<a href="https://colab.research.google.com/github/Heity94/TWSM_Lab/blob/main/Homework/4_Exercise_Representation_2/NL_4_Representation_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers


In [3]:
import pickle
import pandas as pd
import numpy as np
from gensim.models import Word2Vec
from scipy.spatial.distance import cosine
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import tensorflow as tf
import torch
from transformers import BertTokenizer, BertModel
from keras.preprocessing.sequence import pad_sequences
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler


In [4]:
path = '/content/drive/MyDrive/Colab Notebooks/data_lemma.pkl'
with open(path, 'rb') as file:
  df = pickle.load(file)
df = pd.DataFrame(df,columns=['text'])

In [5]:
df.head()

,text
0,car wonder enlighten car see day door sport ca...
1,clock poll final final clock report accelerati...
2,question folk mac plus finally give ghost week...
3,weitek nntp post host amber ssd csd harris com...
4,shuttle launch question article cowcb world st...


In [6]:
corpus_gen = [text.split() for text in df['text']]

**Part 1**

In [14]:
#Initiate Word2Vec model (ignore all words w total freq lower than 566)
model = Word2Vec(corpus_gen, size=100, min_count=566)

In [17]:
#save model
model.save("/content/drive/MyDrive/Colab Notebooks/word2vec.model")

In [23]:
#print sorted features
a = model.wv.vocab.keys()
a

dict_keys(['car', 'wonder', 'see', 'day', 'look', 'early', 'call', 'small', 'body', 'know', 'model', 'year', 'history', 'info', 'mail', 'thank', 'bring', 'report', 'nntp', 'post', 'host', 'washington', 'edu', 'number', 'experience', 'send', 'message', 'speed', 'rate', 'add', 'card', 'disk', 'request', 'network', 'base', 'answer', 'guy', 'question', 'mac', 'give', 'start', 'life', 'way', 'new', 'machine', 'bit', 'maybe', 'anybody', 'expect', 'hear', 'suppose', 'access', 'price', 'line', 'like', 'go', 'display', 'probably', 'get', 'feel', 'well', 'great', 'good', 'opinion', 'people', 'use', 'take', 'size', 'money', 'hit', 'real', 'play', 'figure', 'actually', 'advance', 'email', 'news', 'time', 'truth', 'com', 'uucp', 'write', 'article', 'chip', 'far', 'low', 'level', 'stuff', 'pretty', 'nice', 'require', 'address', 'phone', 'information', 'thing', 'person', 'sense', 'world', 'clear', 'memory', 'error', 'yes', 'quote', 'software', 'check', 'right', 'value', 'set', 'code', 'tell', 'second

In [24]:
print(len(a))
print(len(corpus_gen))
#Word2Vec have filtered more token compared to Exercise 3

435
11314


In [25]:
model.wv['car']
#numpy vector of the word "car" with the predefined size above (100)

array([ 5.62588759e-02,  2.81473160e-01, -2.77809165e-02,  9.52530921e-01,
        7.37310827e-01, -6.21092319e-01, -1.23294592e+00,  1.17196536e+00,
       -1.18000233e+00,  1.34491098e+00,  6.08696900e-02, -8.15124333e-01,
       -1.85230091e-01, -1.00731599e+00, -5.66371202e-01,  6.15894616e-01,
       -8.01912367e-01,  3.46700102e-01, -1.77099109e-01, -1.24831104e+00,
        3.22294116e-01, -4.27762002e-01,  5.26180029e-01, -1.21975195e+00,
       -1.09937274e+00,  1.01581573e+00, -4.99670655e-01,  5.21958590e-01,
        6.81274712e-01, -1.01785553e+00, -3.78396988e-01, -6.82267189e-01,
       -4.50525701e-01,  2.15883803e+00,  1.03632104e+00,  5.22030771e-01,
       -4.73756753e-02,  2.07921839e+00, -4.06512171e-02, -9.01182950e-01,
       -7.51531720e-01,  1.42635095e+00, -1.06533039e+00,  5.39248399e-02,
       -1.21405609e-01,  2.17289791e-01,  1.70174491e+00,  1.53048098e+00,
        2.37964749e-01,  3.70209843e-01,  1.45023882e+00,  8.08004439e-01,
       -2.47110873e-01, -

In [27]:
 model.wv.most_similar('car')
 #makes sense for me overall

[('buy', 0.6046044826507568),
 ('bike', 0.5933898687362671),
 ('friend', 0.5417188405990601),
 ('light', 0.5285630822181702),
 ('figure', 0.4855635166168213),
 ('price', 0.47859400510787964),
 ('get', 0.462393194437027),
 ('pay', 0.4600743353366852),
 ('nice', 0.4591468870639801),
 ('hit', 0.45573240518569946)]

In [30]:
model.wv.most_similar(positive=['bike', 'machine'], topn=1)

[('fast', 0.6857116222381592)]

In [45]:
#find the top 1 most similar words. Positive words contribute positively towards the similarity while negative words negatively.
#This method computes cosine similarity between a simple mean of the projection weight vectors of the given words and the vectors for each word in the model.
model.wv.most_similar(positive=['thank', 'early'], topn=1)
model.wv.most_similar(positive=['see', 'look'], topn=1)
model.wv.most_similar(positive=['armenians', 'turkish'], topn=5)

[('armenian', 0.9483288526535034),
 ('today', 0.5910444259643555),
 ('begin', 0.5887396931648254),
 ('war', 0.5881304144859314),
 ('continue', 0.5836325883865356)]

In [94]:
corpus2 = []
for doc in corpus_gen:
  vec = []
  for word in doc:
    if word in a:
      vec.append(model.wv[word])
    else:
      pass
  if (len(vec) != 0):
    corpus2.append(np.mean(vec,axis=0))
WordtoVecModel = pd.DataFrame(np.concatenate([corpus2],axis=1))
WordtoVecModel.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,-0.202834,0.076092,-0.095726,0.343218,0.158659,-0.292308,-0.419730,0.314719,-0.097812,0.280720,...,-0.092898,-0.236058,-0.391629,0.274717,0.231400,0.032261,0.026370,-0.210911,0.406879,-0.668702
1,0.279975,0.062609,-0.518789,0.439750,-0.054115,-0.357546,-0.125915,0.093503,0.235722,0.310339,...,-0.146856,0.462654,-0.086947,-0.172101,-0.037180,-0.313263,0.065520,-0.245761,0.413866,0.960704
2,0.069948,0.006966,-0.023823,0.466397,-0.282443,-0.033310,0.019614,0.234883,-0.141777,0.045428,...,-0.077648,-0.013572,-0.141621,0.010161,0.264571,0.212483,-0.000674,-0.104642,0.576516,0.060564
3,0.511560,-0.202713,-0.402815,0.405272,0.006115,-0.524349,-0.066496,0.036924,-0.089284,0.146384,...,-0.084344,0.282969,0.068465,0.014603,0.010593,-0.209910,-0.100864,-0.132686,0.089336,0.359928
4,0.327805,-0.518278,-0.254349,0.206288,-0.185592,-0.006242,-0.054440,-0.041821,0.043593,-0.088142,...,-0.099911,0.130097,-0.013865,-0.257857,-0.071710,-0.026071,-0.106727,0.155545,0.245915,0.352365


In [95]:
WordtoVecModel.to_pickle('/content/drive/MyDrive/Colab Notebooks/Models/WordtoVecModel.pkl')

In [105]:
#Find the most similar words to the embedding representing the first document f
v1 = WordtoVecModel.iloc[0].values
model.wv.most_similar(positive=[v1],topn=1)

[('car', 0.863889217376709)]

In [146]:
#Use cosine() to determine the document on the corpus most similar to it.
cosine_lists = []
for i in range(len(WordtoVecModel)-1):
  cosine_list=cosine(v1, WordtoVecModel.iloc[i].values)
  cosine_lists.append(cosine_list)
cosine_lists.pop(0)#eliminate the own element


0.0

In [167]:
#find the closest document with the smallest cosine
#since we popped out the first entry, the actual index of the most similar document is the result + 1
print("The most similar document to the first document is number: ", (np.where(cosine_lists == np.amin(cosine_lists)))[0]+1)


The most similar document to the first document is number:  [4976]


**Part 2**

In [7]:
# Tag Documents
tagged = [TaggedDocument(token, [i]) for i, token in enumerate(corpus_gen)]

#Initialize Doc2Vec Model and train model
model_d = Doc2Vec(tagged, vector_size=100, min_count=566)

In [8]:
model_d.save("/content/drive/MyDrive/Colab Notebooks/doc2vec.model")
model_d.wv.vocab

{'able': <gensim.models.keyedvectors.Vocab at 0x7fce3e8c4fd0>,
 'accept': <gensim.models.keyedvectors.Vocab at 0x7fce3e8c7fd0>,
 'access': <gensim.models.keyedvectors.Vocab at 0x7fce3ebbc250>,
 'act': <gensim.models.keyedvectors.Vocab at 0x7fce3e8ca390>,
 'action': <gensim.models.keyedvectors.Vocab at 0x7fce3e8ca7d0>,
 'actually': <gensim.models.keyedvectors.Vocab at 0x7fce3ebbc850>,
 'add': <gensim.models.keyedvectors.Vocab at 0x7fce44b136d0>,
 'address': <gensim.models.keyedvectors.Vocab at 0x7fce3ebbccd0>,
 'advance': <gensim.models.keyedvectors.Vocab at 0x7fce3ebbc890>,
 'ago': <gensim.models.keyedvectors.Vocab at 0x7fce3e8c75d0>,
 'agree': <gensim.models.keyedvectors.Vocab at 0x7fce3e93d6d0>,
 'allow': <gensim.models.keyedvectors.Vocab at 0x7fce3e93da90>,
 'american': <gensim.models.keyedvectors.Vocab at 0x7fce3e8ca750>,
 'answer': <gensim.models.keyedvectors.Vocab at 0x7fce44b13790>,
 'anybody': <gensim.models.keyedvectors.Vocab at 0x7fce3ebbc2d0>,
 'appear': <gensim.models.keyed

In [13]:
#infer vector
model_d_infer = model_d.infer_vector(tagged[0][0])

In [17]:
model_d.docvecs.most_similar([model_d_infer],topn=10)

[(958, 0.882188618183136),
 (0, 0.7993938326835632),
 (10518, 0.7684688568115234),
 (5371, 0.7506864070892334),
 (7920, 0.7441151142120361),
 (9196, 0.7413545250892639),
 (3819, 0.7409706711769104),
 (3550, 0.7387326955795288),
 (4268, 0.7373998165130615),
 (8346, 0.7246050834655762)]

Here (958, 0.882188618183136) means our provided test_doc is most similar with document 1047 of document set with probability of 88.2%%.

In [16]:
print(tagged[0])
print(tagged[958])

TaggedDocument(['car', 'wonder', 'enlighten', 'car', 'see', 'day', 'door', 'sport', 'car', 'look', 'late', 'early', 'call', 'bricklin', 'door', 'small', 'addition', 'bumper', 'separate', 'rest', 'body', 'know', 'tellme', 'model', 'engine', 'specs', 'year', 'production', 'car', 'history', 'info', 'funky', 'looking', 'car', 'mail', 'thank', 'bring', 'neighborhood', 'lerxst'], [0])
TaggedDocument(['car', 'article', 'apr', 'wam', 'umd', 'edu', 'lerxst', 'wam', 'umd', 'edu', 'thing', 'write', 'wonder', 'enlighten', 'car', 'see', 'day', 'door', 'sport', 'car', 'look', 'late', 'early', 'call', 'bricklin', 'door', 'small', 'addition', 'bumper', 'separate', 'rest', 'body', 'know', 'tellme', 'model', 'engine', 'specs', 'year', 'production', 'car', 'history', 'info', 'funky', 'looking', 'car', 'mail', 'bricklin', 'manufacture', 'engine', 'ford', 'odd', 'look', 'encase', 'bumper', 'aren', 'lot', 'hemming', 'motor', 'news', 'ususally', 'list', 'basically', 'performance', 'ford', 'new', 'styling', '

In [25]:
#create DataFrame and store in File
corpus_gen_d = [model_d.infer_vector(doc) for doc, id in tagged]
DoctoVecModel = pd.DataFrame(np.concatenate([corpus_gen_d],axis=0))
DoctoVecModel.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,-0.053619,0.011653,-0.088086,0.009998,0.058123,0.057391,-0.028622,0.072953,0.000197,0.013468,...,0.020359,-0.026667,-0.023817,-0.040381,-0.072048,0.067814,0.112089,0.062354,0.045778,-0.011178
1,-0.010907,0.004340,-0.026339,0.019340,0.006540,0.068311,0.028034,-0.009090,-0.005152,-0.003086,...,-0.009773,0.019701,-0.031870,-0.022935,0.047459,-0.013812,0.053801,-0.048401,0.082410,-0.003129
2,-0.096821,0.029188,-0.088032,-0.064737,0.081996,-0.063273,-0.039238,-0.079276,0.074540,0.027597,...,-0.151085,-0.017132,-0.079529,0.028721,0.080191,-0.104284,-0.005967,0.053793,-0.019547,0.043546
3,0.010929,-0.023399,0.015611,-0.074644,0.014051,0.008594,-0.021287,-0.066496,0.012437,0.067700,...,0.048647,-0.084668,0.074614,0.027904,-0.036989,-0.030744,0.042501,-0.007357,0.008964,0.017559
4,-0.048285,0.008263,-0.046788,-0.076970,0.038844,0.120856,-0.015918,-0.049009,-0.026472,0.055862,...,-0.014931,-0.032638,0.064078,-0.030750,0.069489,0.025870,0.115464,-0.034172,0.082544,0.056793


In [26]:
#store into pickle 
DoctoVecModel.to_pickle('/content/drive/MyDrive/Colab Notebooks/Models/DoctoVecModel.pkl')